In [1]:
import warnings
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer, transpile, IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
warnings.filterwarnings("ignore", category=DeprecationWarning)
import numpy as np
import pandas as pd
import math

from qiskit.quantum_info import Statevector

from scipy.stats import chi2_contingency, ttest_ind

import unittest
import hypothesis.strategies as st
from hypothesis import given, settings

pi = np.pi

In [2]:
def measure_z(circuit, qubit_indexes):
    cBitIndex = 0
    for index in qubit_indexes:
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

def measure_x(circuit, qubitIndexes):
    cBitIndex = 0
    for index in qubitIndexes:
        circuit.h(index)
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

def measure_y(circuit, qubit_indexes):
    cBitIndex = 0
    for index in qubit_indexes:
        circuit.sdg(index)
        circuit.h(index)
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

In [3]:
# Completed but more testing required
def assertEntangled(backend,qc,qubit1,qubit2,measurements_to_make,alpha = 0.05):
    if (qc.num_clbits == 0):
        qc.add_register(ClassicalRegister(2))
    elif (qc.num_clbits != 2):
        raise ValueError("QuantumCircuit classical register must be of length 2")
    
    zQuantumCircuit = measure_z(qc, [qubit1, qubit2])
    zJob = execute(zQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    zMemory = zJob.result().get_memory()
    zCounts = zJob.result().get_counts()
    # zDf = pd.DataFrame(columns=['q0', 'q1'])
    # for row in zCounts:
    #     zDf = zDf.append({'q0':row[0], 'q1':row[1]}, ignore_index=True)
    # print(zDf.astype(str))
    resDf = pd.DataFrame(columns=['0','1'])
    classical_qubit_index = 1
    for qubit in [qubit1,qubit2]:
        zero_amount, one_amount = 0,0
        for experiment in zCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                zero_amount += zCounts[experiment]
            else:
                one_amount += zCounts[experiment]
        df = {'0':zero_amount, '1':one_amount}
        resDf = resDf.append(df, ignore_index = True)
        classical_qubit_index+=1

    resDf['0'] = resDf['0'].astype(int)
    resDf['1'] = resDf['1'].astype(int)

    print(resDf.astype(str))
    chiVal, pVal, dOfFreedom, exp = chi2_contingency(resDf)
    print("chi square value: ",chiVal,"p value: ",pVal,"expected values: ",exp)
    if(pVal<alpha):
        raise(AssertionError("states are not entangled"))
    else:
        print("states are entangled")


backend = Aer.get_backend('aer_simulator')
qc3 = QuantumCircuit(2)
qc3.h(0)
qc3.cnot(0,1)
# qc3.x(0)
# qc3.h(0)
# qc3.cnot(0,1)
# qc3.h(1)
# qc3.h(0)
assertEntangled(backend,qc3,0,1,100000,0.05)

       0      1
0  49699  50301
1  49699  50301
chi square value:  0.0 p value:  1.0 expected values:  [[49699. 50301.]
 [49699. 50301.]]
states are entangled


In [4]:
# Completed but more testing required
## assert that qubits are equal
def assertEqual(backend, quantumCircuit, qubits_to_assert, measurements_to_make, alpha):
    ## needs to make at least 2 measurements, one for x axis, one for y axis
    ## realistically we need more for any statistical significance
    if (measurements_to_make < 2):
        raise ValueError("Must make at least 2 measurements")

    # makes sure qubits_to_assert is a list
    if (not isinstance(qubits_to_assert, list)):
        qubits_to_assert = [qubits_to_assert]

    ## classical register must be of same length as amount of qubits to assert
    ## if there is no classical register add them according to length of qubit list
    if (quantumCircuit.num_clbits == 0):
        quantumCircuit.add_register(ClassicalRegister(len(qubits_to_assert)))
    elif (len(qubits_to_assert) != 2):
        raise ValueError("QuantumCircuit classical register must be of length 2")

    ## divide measurements to make by 3 as we need to run measurements twice, one for x and one for y
    measurements_to_make = measurements_to_make // 3

    ## copy the circit and set measurement to y axis
    yQuantumCircuit = measure_y(quantumCircuit.copy(), qubits_to_assert)

    ## measure the x axis
    xQuantumCircuit = measure_x(quantumCircuit.copy(), qubits_to_assert)

    ## measure the z axis
    zQuantumCircuit = measure_z(quantumCircuit, qubits_to_assert)

    ## get y axis results
    yJob = execute(yQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    yMemory = yJob.result().get_memory()
    yCounts = yJob.result().get_counts()

    ## get x axis results
    xJob = execute(xQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    xMemory = xJob.result().get_memory()
    xCounts = xJob.result().get_counts()

    ## get z axis results
    zJob = execute(zQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    zMemory = zJob.result().get_memory()
    zCounts = zJob.result().get_counts()

    resDf = pd.DataFrame(columns=['0','1','+','i','-','-i'])

    classical_qubit_index = 1
    for qubit in qubits_to_assert:
        zero_amount, one_amount, plus_amount, i_amount, minus_amount, minus_i_amount = 0,0,0,0,0,0
        for experiment in xCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                plus_amount += xCounts[experiment]
            else:
                minus_amount += xCounts[experiment]
        for experiment in yCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                i_amount += yCounts[experiment]
            else:
                minus_i_amount += yCounts[experiment]
        for experiment in zCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                zero_amount += zCounts[experiment]
            else:
                one_amount += zCounts[experiment]
        df = {'0':zero_amount, '1':one_amount,
              '+':plus_amount, 'i':i_amount,
              '-':minus_amount,'-i':minus_i_amount}
        resDf = resDf.append(df, ignore_index = True)
        classical_qubit_index+=1

    ## convert the columns to a strict numerical type
    resDf['+'] = resDf['+'].astype(int)
    resDf['i'] = resDf['i'].astype(int)
    resDf['-'] = resDf['-'].astype(int)
    resDf['-i'] = resDf['-i'].astype(int)
    resDf['0'] = resDf['0'].astype(int)
    resDf['1'] = resDf['1'].astype(int)
    print(resDf.astype(str))
    # print(z1)
    arr = resDf.to_numpy()
    q1Vals = arr[0, 0:] # Stores measurements across all axes for the 1st qubit
    print("Measurements for qubit1: ", q1Vals)
    q2Vals = arr[1, 0:] # Stores measurements across all axes for the 2nd qubit
    print("Measurements for qubit2: ", q2Vals)

    tTest, pValue = ttest_ind(q1Vals, q2Vals, alternative = 'two-sided') # Apply t test
    print("stat: ",tTest, "pValue: ", pValue)
    if pValue > alpha:
        print("The two qubits are equal (fail to reject null hypothesis) ")
    else:
        print("There is a significant difference between the two qubits (reject null hypothesis)")

qc = QuantumCircuit(2)
# qc.initialize([0, 1/np.sqrt(2), -1.j/np.sqrt(2), 0], qc.qubits)
# qc.h(0)
# qc.x(1)
# qc.p(0.5*2*math.pi/100, 1)
# qc.h(0)
# qc.h(1)
# qc.p(10*2*math.pi/100, 0)
# qc.p(20*2*math.pi/100, 1)
assertEqual(backend, qc, [0,0], 300000, 0.01)

        0  1      +      i      -     -i
0  100000  0  49639  50088  50361  49912
1  100000  0  50102  50003  49898  49997
Measurements for qubit1:  [100000      0  49639  50088  50361  49912]
Measurements for qubit2:  [100000      0  50102  50003  49898  49997]
stat:  0.0 pValue:  1.0
The two qubits are equal (fail to reject null hypothesis) 


In [54]:
# Almost complete still testing

def assertProbability(qc,qubitChoice: int, qubitState :str, expectedProbability):
    sv = Statevector.from_label("00")
    evl = sv.evolve(qc)
    probs = evl.probabilities_dict([qubitChoice])
    probsRound = {key: round(values,2) for key,values in probs.items()}
    
    for key,value in probsRound.items():
        if(key==qubitState):
            if(value==expectedProbability):
                print("Expected Probability present")
                return True
            else:
                raise(AssertionError("Probability not present"))
    raise(AssertionError("Probability not present"))
            

qc = QuantumCircuit(2)
qc.h(0)
assertProbability(qc,1,"0",1)

Expected Probability present


True